In [1]:
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")

import numpy as np
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt   

import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [6]:
merged_data=pd.read_csv('../data/merged_data.csv')

merged_data

,ISIN,Name,Mkt Cap,Industry,BI350,BI493,EG056,EG057,ES018,ES019,...,SR231,SR249,ST024,SR752,SR754,SR756,SR760,SR724,SR721,Year
0,CNE100004ZD8,ACM RESEARCH S-A,"4,885,443,926.00",Semiconductors,0.0,No data,25.644,2.219,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
1,US00108J1097,ACM RESEARCH-A,"1,474,910,952.00",Semiconductors,0.0,No data,120.409,47.224,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
2,US0079031078,ADV MICRO DEVICE,"241,546,435,400.00",Semiconductors,0.0,No data,385.733,37.01,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
3,TW0003438008,ADVANCED ANALOG,"98,788,154.60",Semiconductors,0.0,No data,33.397,7.594,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
4,CNE100003MM9,ADVANCED MICRO-A,"11,573,338,720.00",Semiconductors,0.0,100,64.797,4.375,No data,0.023,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,US0395872098,ARCIMOTO INC,"1,172,755.33",Automobiles,0.0,No data,1353.663,486.827,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
344,US54405Q2093,NU RIDE INC,"24,087,261.50",Automobiles,0.0,No data,1754.48,80.799,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
345,US3073597037,FARADAY FUTURE I,"167,343,808.45",Automobiles,0.0,No data,160.608,0.829,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023
346,CA35954F1080,FUELPOSITIVE COR,"19,271,698.47",Automobiles,0.0,No data,No data,597.368,No data,No data,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2023


In [7]:
df_materiality=pd.read_csv('../data/raw/materiality.csv')


In [8]:
df_materiality = df_materiality.fillna('0')
df_materiality = df_materiality.drop('Field Name', axis=1)


In [9]:
for column in df_materiality.columns:
    try:
        df_materiality[column] = pd.to_numeric(df_materiality[column], errors='raise')
    except ValueError:
        pass 

df_materiality.head()

,Type,Issue,Score Field ID,Exploration & Production,Integrated Oils,Midstream,Refining & Marketing,Drilling & Drilling Support,Oilfield Services & Equipment,"Internet Media, Platforms and Services",...,Wind Energy Equipment,Renewable Energy Project Developers,"Technology Hardware, Equipment and Components",EMS/ODM,Semiconductors,Software and Tech Services,Integrated Electric Utilities and Power Generation,Electric Transmission & Dist,Gas Utilities,Water Utilities
0,Environmental,Air Quality,SR208,Y,Y,Y,Y,Y,Y,0,...,0,0,0,0,0.0,0,Y,0,0,0
1,Environmental,Climate Exposure,SR235,Y,Y,0,0,0,0,0,...,0,0,0,0,0.0,0,Y,Y,0,Y
2,Environmental,Ecological Impact,SR231,Y,Y,Y,Y,Y,Y,0,...,Y,Y,0,0,0.0,0,Y,0,Y,Y
3,Environmental,Energy Management,SR205,0,Y,0,0,Y,Y,Y,...,0,0,0,0,0.3,Y,0,0,0,Y
4,Environmental,Environmental Supply Chain Management,SR249,0,0,0,0,0,0,0,...,Y,0,0,0,0.0,0,0,0,0,0


In [10]:
categories = df_materiality['Issue'].tolist()

# Crear un nuevo DataFrame vacío para almacenar los resultados
results = pd.DataFrame(columns=['Industry', 'type', 'ISIN'] + categories)

# Iterar a través de merged_data
for index, row in merged_data.iterrows():
    industry = row['Industry']
    isin = row['ISIN']
    
    # Crear filas base para los resultados
    result_row_env = [industry, 'Environmental', isin] + [0]*len(categories)
    result_row_soc = [industry, 'Social', isin] + [0]*len(categories)
    result_row_gov = [industry, 'Governance', isin] + [0]*len(categories)

    for index2, row2 in df_materiality.iterrows():
        score_id = row2['Score Field ID']
        category_type = row2['Type']
        issue = row2['Issue']
        
        if score_id in row.index:
            try:
                value = float(row[score_id])
            except ValueError:
                value = 0
            
            # Asignar valores a las categorías correspondientes
            if category_type == 'Environmental':
                result_row_env[3 + categories.index(issue)] = value
            elif category_type == 'Social':
                result_row_soc[3 + categories.index(issue)] = value
            elif category_type == 'Governance':
                result_row_gov[3 + categories.index(issue)] = value

    # Convertir listas a Series para agregar al DataFrame de resultados
    result_row_env = pd.Series(result_row_env, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_soc = pd.Series(result_row_soc, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_gov = pd.Series(result_row_gov, index=['Industry', 'type', 'ISIN'] + categories)

    # Agregar las filas al DataFrame de resultados usando concat
    results = pd.concat([results, result_row_env.to_frame().T, result_row_soc.to_frame().T, result_row_gov.to_frame().T], ignore_index=True)


results

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,4.816013,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Semiconductors,Social,CNE100004ZD8,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
2,Semiconductors,Governance,CNE100004ZD8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.781484,1.60677,7.086886,7.334174
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Semiconductors,Social,US00108J1097,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Automobiles,Social,CA35954F1080,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1040,Automobiles,Governance,CA35954F1080,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1042,Automobiles,Social,SGXZ55111462,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [11]:
for index, row in results.iterrows():
    industry = row['Industry']
    
    # Verificar si industry no es NaN
    if pd.notna(industry):
        # Acceder al multiplicador solo si industry es válido
        if industry in df_materiality.columns:
            multiplier = df_materiality[industry].values
            
            # Identificar las columnas a multiplicar (ajustar los nombres de columna según la estructura real de tu DataFrame)
            columns_to_multiply = results.columns[3:]  # Ajustar para seleccionar las columnas que deseas multiplicar
            
            # Realizar la multiplicación para las columnas seleccionadas
            results.loc[index, columns_to_multiply] *= multiplier
        else:
            print(f"Industry '{industry}' no está en df_materiality.")
    else:
        print("Warning: 'Industry' value is NaN for row index", index)
results
        

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.721334,0.385625,1.700853,1.173468
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Automobiles,Social,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1040,Automobiles,Governance,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1042,Automobiles,Social,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:

results['ENVIRONMENTAL SCORE'] = results[results['type'] == 'Environmental'].iloc[:, 3:].sum(axis=1)
results['SOCIAL SCORE'] = results[results['type'] == 'Social'].iloc[:, 3:].sum(axis=1)
results['GOVERNANCE SCORE'] = results[results['type'] == 'Governance'].iloc[:, 3:].sum(axis=1)

grouped = results.groupby('ISIN').sum()

grouped['ESG SCORE'] = (grouped['ENVIRONMENTAL SCORE'] * 0.4 + 
                        grouped['SOCIAL SCORE'] * 0.3 + 
                        grouped['GOVERNANCE SCORE'] * 0.3)

# Unir los puntajes ESG de vuelta a los datos originales
results = results.merge(grouped[['ESG SCORE']], on='ISIN', how='left')

In [13]:
results

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.06801,NaN,NaN,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.08,NaN,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,NaN,NaN,4.981279,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.08,NaN,2.030903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Automobiles,Social,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
1040,Automobiles,Governance,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1042,Automobiles,Social,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0


In [14]:
results['ENVIRONMENTAL SCORE'] = results['ENVIRONMENTAL SCORE'].fillna(0)
results['SOCIAL SCORE'] = results['SOCIAL SCORE'].fillna(0)
results['GOVERNANCE SCORE'] = results['GOVERNANCE SCORE'].fillna(0)
results['ESG SCORE'] = results['ESG SCORE'].fillna(0)
results

C:\Users\Veronica\AppData\Local\Temp\ipykernel_53940\2142310382.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results['ENVIRONMENTAL SCORE'] = results['ENVIRONMENTAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_53940\2142310382.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results['SOCIAL SCORE'] = results['SOCIAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_53940\2142310382.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version.

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.06801,0.00,0.000000,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,1.08,0.000000,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,0.00000,0.00,4.981279,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.000000,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,1.08,0.000000,2.030903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Automobiles,Social,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.000000,0.000000
1040,Automobiles,Governance,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.000000,0.000000
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.000000,0.000000
1042,Automobiles,Social,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.000000,0.000000


In [15]:
results[results['ESG SCORE']==0]


,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
969,Semiconductors,Environmental,KR7030530000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970,Semiconductors,Social,KR7030530000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
971,Semiconductors,Governance,KR7030530000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1035,Automobiles,Environmental,US3073597037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036,Automobiles,Social,US3073597037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1037,Automobiles,Governance,US3073597037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1038,Automobiles,Environmental,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1039,Automobiles,Social,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1040,Automobiles,Governance,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
for column in results.columns:
    try:
        results[column] = pd.to_numeric(results[column], errors='raise')
    except ValueError:
        pass 

results.head()

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.06801,0.00,0.000000,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,1.08,0.000000,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,0.00000,0.00,4.981279,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,1.08,0.000000,2.030903


In [17]:
Enviromental =results[results['type'] == 'Environmental']
Social =results[results['type'] == 'Social']
Governance =results[results['type'] == 'Governance']


In [18]:
columns_E= ['Access & Affordability',
 'Community Rights & Relations',
 'Customer Welfare',
 'Data Security & Customer Privacy',
 'Ethics & Compliance',
 'Labor & Employment Practices',
 'Marketing & Labeling',
 'Occupational Health & Safety Management',
 'Operational Risk Management',
 'Product Quality Management',
 'Social Supply Chain Management',
 'Board Composition',
 'Executive Compensation',
 'Shareholder Rights',
 'Audit',
'SOCIAL SCORE',
 'GOVERNANCE SCORE',
 'ESG SCORE']
columns_S= ['Air Quality',
 'Climate Exposure',
 'Ecological Impact',
 'Energy Management',
 'Environmental Supply Chain Management',
 'GHG Emissions Management',
 'Sustainable Finance',
 'Sustainable Product',
 'Waste Management',
 'Water Management', 'Board Composition',
 'Executive Compensation',
 'Shareholder Rights',
 'Audit',
'ENVIRONMENTAL SCORE',          
 'GOVERNANCE SCORE',
 'ESG SCORE']
columns_G= [
 'Air Quality',
 'Climate Exposure',
 'Ecological Impact',
 'Energy Management',
 'Environmental Supply Chain Management',
 'GHG Emissions Management',
 'Sustainable Finance',
 'Sustainable Product',
 'Waste Management',
 'Water Management',
 'Access & Affordability',
 'Community Rights & Relations',
 'Customer Welfare',
 'Data Security & Customer Privacy',
 'Ethics & Compliance',
 'Labor & Employment Practices',
 'Marketing & Labeling',
 'Occupational Health & Safety Management',
 'Operational Risk Management',
 'Product Quality Management',
 'Social Supply Chain Management',
 'ENVIRONMENTAL SCORE',
 'SOCIAL SCORE',
 'ESG SCORE']


Enviromental=Enviromental.drop(columns=columns_E)
Social=Social.drop(columns=columns_S)
Governance=Governance.drop(columns=columns_G)


In [19]:
cols_to_sum = results.columns[3:-1]  # Excluye las primeras 3 columnas y 'GLOBAL SCORE'

sum_results = results.groupby('Industry')[cols_to_sum].sum()

unique_isins_per_industry = results.groupby('Industry')['ISIN'].nunique()

mean_results = sum_results.div(unique_isins_per_industry, axis=0)

mean_results['ESG SCORE'] = results.groupby('Industry')['ESG SCORE'].mean()

In [20]:
mean_results

,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
Industry,,,,,,,,,,,,,,,,,,,,,
Automobiles,0.0,0.27052,0.0,0.000000,0.0,0.457351,0.0,1.050969,0.466498,0.000000,...,1.159411,0.0,1.588574,0.871988,0.904179,1.060332,2.245338,2.364194,4.425073,2.934915
Semiconductors,0.0,0.00000,0.0,0.967335,0.0,0.366032,0.0,0.123263,0.347286,1.141484,...,0.360398,0.0,1.755816,1.066333,1.585308,1.113278,2.945400,3.101457,5.520735,3.764817


In [21]:
Enviromental['Mean Environmental'] = Enviromental.groupby('Industry')['ENVIRONMENTAL SCORE'].transform('mean')
Social['Mean Social'] = Social.groupby('Industry')['SOCIAL SCORE'].transform('mean')
Governance['Mean Governance'] = Governance.groupby('Industry')['GOVERNANCE SCORE'].transform('mean')


In [22]:
Enviromental.head()

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,ENVIRONMENTAL SCORE,Mean Environmental
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.000000,0.0,0.00,0.550956,1.072250,3.068010,2.9454
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,2.9454
6,Semiconductors,Environmental,US0079031078,0.0,0.0,0.0,2.446815,0.0,0.767124,0.0,0.48,1.067321,2.564379,7.325640,2.9454
9,Semiconductors,Environmental,TW0003438008,0.0,0.0,0.0,0.000000,0.0,0.044511,0.0,0.00,0.000000,1.674717,1.719228,2.9454
12,Semiconductors,Environmental,CNE100003MM9,0.0,0.0,0.0,1.370178,0.0,0.044511,0.0,0.48,0.841408,1.445758,4.181856,2.9454


In [23]:
Social.head()

,Industry,type,ISIN,Access & Affordability,Community Rights & Relations,Customer Welfare,Data Security & Customer Privacy,Ethics & Compliance,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,SOCIAL SCORE,Mean Social
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.080000,0.0,0.00,0.0,1.080000,3.101457
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.080000,0.0,0.00,0.0,1.080000,3.101457
7,Semiconductors,Social,US0079031078,0.0,0.3,0.0,0.0,1.510403,0.983832,0.0,3.085222,0.0,0.00,0.0,5.879457,3.101457
10,Semiconductors,Social,TW0003438008,0.0,0.3,0.0,0.0,0.208524,0.000000,0.0,1.080000,0.0,0.78,0.0,2.368524,3.101457
13,Semiconductors,Social,CNE100003MM9,0.0,0.3,0.0,0.0,1.900000,0.857878,0.0,1.080000,0.0,0.00,0.0,4.137878,3.101457


In [24]:
Governance.head()

,Industry,type,ISIN,Board Composition,Executive Compensation,Shareholder Rights,Audit,GOVERNANCE SCORE,Mean Governance
2,Semiconductors,Governance,CNE100004ZD8,1.721334,0.385625,1.700853,1.173468,4.981279,5.520735
5,Semiconductors,Governance,US00108J1097,2.425561,1.300311,0.652884,1.310922,5.689677,5.520735
8,Semiconductors,Governance,US0079031078,2.635831,1.902486,1.699084,1.317092,7.554493,5.520735
11,Semiconductors,Governance,TW0003438008,2.012439,0.947413,1.770664,1.208414,5.938929,5.520735
14,Semiconductors,Governance,CNE100003MM9,0.902518,0.845444,1.700853,0.966476,4.415290,5.520735


In [25]:
ESG = results[['ISIN','Industry', 'ESG SCORE']].drop_duplicates().reset_index(drop=True)


In [26]:
score1 = Enviromental[['ISIN', 'ENVIRONMENTAL SCORE']]
score2 = Social[['ISIN', 'SOCIAL SCORE']]
score3 = Governance[['ISIN', 'GOVERNANCE SCORE']]

ESG = pd.merge(ESG, score1, on='ISIN', how='left')
ESG = pd.merge(ESG, score2, on='ISIN', how='left')
ESG = pd.merge(ESG, score3, on='ISIN', how='left')

# Definir los bins y las etiquetas correspondientes
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8', '8-9', '9-10']

# Crear la nueva columna 'ESG Score Range' en el DataFrame ESG
ESG['ESG Score Range'] = pd.cut(ESG['ESG SCORE'], bins=bins, labels=labels, right=False)

ESG['Mean ESG'] = ESG.groupby('Industry')['ESG SCORE'].transform('mean')



In [27]:
def clasificar_esg(esg_score, median_esg):
    esg_score_int = int(round(esg_score))
    median_esg_int = int(round(median_esg))
    if esg_score_int > median_esg_int:
        return 'Above Median'
    elif esg_score_int == median_esg_int:
        return 'Leading'
    else:
        return 'Below Median'

# Crear la nueva columna 'ESG Classification'
ESG['ESG Classification'] = ESG.apply(lambda row: clasificar_esg(row['ESG SCORE'], ESG.groupby('Industry')['ESG SCORE'].transform('median')[row.name]), axis=1)

# Crear las nuevas columnas de clasificación sin crear columnas intermedias para las medianas
ESG['E Classification'] = ESG.apply(lambda row: clasificar_esg(row['ENVIRONMENTAL SCORE'], ESG.groupby('Industry')['ENVIRONMENTAL SCORE'].transform('median')[row.name]), axis=1)
ESG['S Classification'] = ESG.apply(lambda row: clasificar_esg(row['SOCIAL SCORE'], ESG.groupby('Industry')['SOCIAL SCORE'].transform('median')[row.name]), axis=1)
ESG['G Classification'] = ESG.apply(lambda row: clasificar_esg(row['GOVERNANCE SCORE'], ESG.groupby('Industry')['GOVERNANCE SCORE'].transform('median')[row.name]), axis=1)

ESG.head()

,ISIN,Industry,ESG SCORE,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG Score Range,Mean ESG,ESG Classification,E Classification,S Classification,G Classification
0,CNE100004ZD8,Semiconductors,3.045588,3.068010,1.080000,4.981279,3-4,3.764817,Below Median,Leading,Below Median,Leading
1,US00108J1097,Semiconductors,2.030903,0.000000,1.080000,5.689677,2-3,3.764817,Below Median,Below Median,Below Median,Above Median
2,US0079031078,Semiconductors,6.960441,7.325640,5.879457,7.554493,6-7,3.764817,Above Median,Above Median,Above Median,Above Median
3,TW0003438008,Semiconductors,3.179927,1.719228,2.368524,5.938929,3-4,3.764817,Below Median,Below Median,Below Median,Above Median
4,CNE100003MM9,Semiconductors,4.238693,4.181856,4.137878,4.415290,4-5,3.764817,Leading,Above Median,Above Median,Below Median


In [28]:
materiality=df_materiality.T
new_header = materiality.iloc[1]  # Seleccionar la tercera fila como nueva cabecera#
materiality = materiality[3:]  # Saltar las primeras tres filas y usar el resto como datos
materiality.columns = new_header


In [29]:
def get_zero_columns(row):
    zero_columns = row[(row == '0') | (row == 0)].index.tolist()
    return zero_columns
materiality['Zero Columns'] = materiality.apply(get_zero_columns, axis=1)
materiality=materiality[['Zero Columns']]
materiality=materiality.reset_index()
materiality = materiality.rename(columns={'index': 'Industry'})
materiality.head()

Issue,Industry,Zero Columns
0,Exploration & Production,"[Energy Management, Environmental Supply Chain..."
1,Integrated Oils,"[Environmental Supply Chain Management, Sustai..."
2,Midstream,"[Climate Exposure, Energy Management, Environm..."
3,Refining & Marketing,"[Climate Exposure, Energy Management, Environm..."
4,Drilling & Drilling Support,"[Climate Exposure, Environmental Supply Chain ..."


In [30]:
for index, row in Enviromental.iterrows():
    industry = row['Industry']
    
    # Buscar coincidencias en materiality basado en Industry
    mask = materiality['Industry'] == industry
    
    if mask.any():
        # Obtener las Zero Columns correspondientes
        zero_columns = materiality.loc[mask, 'Zero Columns'].iloc[0]
        
      
        for column in zero_columns:
            if column in Enviromental.columns:
                Enviromental.at[index, column] = np.nan

Enviromental.head()


,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,ENVIRONMENTAL SCORE,Mean Environmental
0,Semiconductors,Environmental,CNE100004ZD8,NaN,NaN,NaN,1.444804,NaN,0.000000,NaN,0.00,0.550956,1.072250,3.068010,2.9454
3,Semiconductors,Environmental,US00108J1097,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.00,0.000000,0.000000,0.000000,2.9454
6,Semiconductors,Environmental,US0079031078,NaN,NaN,NaN,2.446815,NaN,0.767124,NaN,0.48,1.067321,2.564379,7.325640,2.9454
9,Semiconductors,Environmental,TW0003438008,NaN,NaN,NaN,0.000000,NaN,0.044511,NaN,0.00,0.000000,1.674717,1.719228,2.9454
12,Semiconductors,Environmental,CNE100003MM9,NaN,NaN,NaN,1.370178,NaN,0.044511,NaN,0.48,0.841408,1.445758,4.181856,2.9454


In [31]:
for index, row in Social.iterrows():
    industry = row['Industry']
    
    # Buscar coincidencias en materiality basado en Industry
    mask = materiality['Industry'] == industry
    
    if mask.any():
        # Obtener las Zero Columns correspondientes
        zero_columns = materiality.loc[mask, 'Zero Columns'].iloc[0]
        
      
        for column in zero_columns:
            if column in Social.columns:
                Social.at[index, column] = np.nan

Social.head()


,Industry,type,ISIN,Access & Affordability,Community Rights & Relations,Customer Welfare,Data Security & Customer Privacy,Ethics & Compliance,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,SOCIAL SCORE,Mean Social
1,Semiconductors,Social,CNE100004ZD8,NaN,0.0,NaN,NaN,0.000000,0.000000,NaN,1.080000,NaN,0.00,NaN,1.080000,3.101457
4,Semiconductors,Social,US00108J1097,NaN,0.0,NaN,NaN,0.000000,0.000000,NaN,1.080000,NaN,0.00,NaN,1.080000,3.101457
7,Semiconductors,Social,US0079031078,NaN,0.3,NaN,NaN,1.510403,0.983832,NaN,3.085222,NaN,0.00,NaN,5.879457,3.101457
10,Semiconductors,Social,TW0003438008,NaN,0.3,NaN,NaN,0.208524,0.000000,NaN,1.080000,NaN,0.78,NaN,2.368524,3.101457
13,Semiconductors,Social,CNE100003MM9,NaN,0.3,NaN,NaN,1.900000,0.857878,NaN,1.080000,NaN,0.00,NaN,4.137878,3.101457


In [32]:

Enviromental.to_csv('../data/enviromental.csv', index=False)
Social.to_csv('../data/social.csv', index=False)
Governance.to_csv('../data/governance.csv', index=False)

In [33]:
ESG.to_csv('../data/ESG_score.csv', index=False)

In [34]:
esg_percentage = ESG.groupby(['Industry', 'ESG Score Range']).size().reset_index(name='Count')
total_count = ESG.groupby('Industry').size().reset_index(name='Total')
esg_percentage = esg_percentage.merge(total_count, on='Industry')
esg_percentage['Percentage'] = (esg_percentage['Count'] / esg_percentage['Total']) * 100
esg_percentage


C:\Users\Veronica\AppData\Local\Temp\ipykernel_53940\3098163376.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  esg_percentage = ESG.groupby(['Industry', 'ESG Score Range']).size().reset_index(name='Count')


,Industry,ESG Score Range,Count,Total,Percentage
0,Automobiles,0-1,3,17,17.647059
1,Automobiles,1-2,2,17,11.764706
2,Automobiles,2-3,3,17,17.647059
3,Automobiles,3-4,7,17,41.176471
4,Automobiles,4-5,0,17,0.000000
5,Automobiles,5-6,1,17,5.882353
6,Automobiles,6-7,1,17,5.882353
7,Automobiles,7-8,0,17,0.000000
8,Automobiles,8-9,0,17,0.000000
9,Automobiles,9-10,0,17,0.000000


In [35]:
esg_percentage.to_csv('../data/esg_percentage.csv', index=False)